In [1]:
# --- 安装必要的库 ---
# 请确保您已按照上述说明安装了 libvips 系统库
# pip install pyvips opencv-python scikit-image tqdm

import os
import cv2
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from skimage.color import rgb2hed
from skimage.filters import threshold_otsu
from IPython.display import display, Image as IPImage

# 导入 pyvips 库
try:
    import pyvips
except ImportError:
    print("错误: pyvips 库未找到。请按照单元格上方的说明进行安装。")
    # 在 Notebook 中，我们可以用 raise 来停止执行
    raise

# 使用 tqdm.notebook 可以在 Notebook 中显示更好看的进度条
from tqdm.auto import tqdm

os.chdir("/home1/jijh/diffusion_project/ADiffusion")
os.getcwd()


'/public/home/jijh/diffusion_project/ADiffusion'

In [ ]:
import os
import scanpy as sc
import pandas as pd
import numpy as np
from tqdm.auto import tqdm
from scipy.sparse import issparse

# 导入您自定义的库
from src.pipeline.hest_loading import HESTDataset

def load_sample_data(sample, batch_key: str = 'sample_id'):
    """Helper function to load a single sample's data"""
    try:
        sample.load_st_data(lazy=False)
        if sample.adata is not None:
            # 确保每个adata有唯一的obs_names以避免冲突
            sample.adata.obs_names = [f"{sample.sample_id}_{obs_name}" for obs_name in sample.adata.obs_names]
            # 添加批次信息
            sample.adata.obs[batch_key] = sample.sample_id
            return sample.adata.to_memory()
        return None
    except Exception as e:
        print(f"Error loading sample {sample.sample_id}: {e}")
        return None

# --- 函数定义 ---

def calculate_global_hvgs(
    samples_to_process: list, 
    n_top_genes: int = 3000, 
    batch_key: str = 'sample_id',
    sentence_output_basedir_hvg: str = "output/hvgs"
    
) -> pd.Index:
    """
    合并多个AnnData对象，进行批次校正，并计算全局高变基因。

    参数:
        samples_to_process (list): HESTSample对象列表。
        n_top_genes (int): 要识别的高变基因数量。
        batch_key (str): 用于批次校正的adata.obs中的列名。

    返回:
        pd.Index: 包含高变基因名称的索引。
    """
    print("开始合并所有样本以计算全局HVGs...")
    
    # Load data sequentially instead of using multiprocessing
    adatas = []
    
    for sample in tqdm(samples_to_process, desc="加载并合并AnnData"):
        adata = load_sample_data(sample, batch_key)
        if adata is not None:
            adatas.append(adata)

    if not adatas:
        raise ValueError("未能加载任何AnnData对象。")

    # 合并前检查并处理重复基因名
    processed_adatas = []
    for adata in adatas:
        if adata.var_names.duplicated().any():
            print(f"发现重复基因名，正在处理样本中的 {adata.var_names.duplicated().sum()} 个重复基因...")
            # 保留第一次出现的基因，删除后续重复的
            adata = adata[:, ~adata.var_names.duplicated()].copy()
        processed_adatas.append(adata)
    
    # 合并所有AnnData对象
    # uns_merge='unique' 避免合并时因 .uns 字典中的冲突而报错
    adata_merged = sc.concat(processed_adatas, join='outer', uns_merge='unique')
    print(f"所有样本已合并。总尺寸: {adata_merged.shape}")
    
    # 标准化流程
    sc.pp.normalize_total(adata_merged, target_sum=1e4)
    sc.pp.log1p(adata_merged)
    
    # 计算初始的HVGs（用于后续PCA和Harmony）
    sc.pp.highly_variable_genes(adata_merged, n_top_genes=n_top_genes, batch_key=batch_key)
    
    # 提取高变基因列表
    hvg_list = adata_merged.var.index[adata_merged.var['highly_variable']]
    # Write the HVG list to a file
    hvg_file_path = os.path.join(sentence_output_basedir_hvg, "global_hvgs.txt")
    print(f"成功识别出 {len(hvg_list)} 个全局高变基因。")
    
    return hvg_list


In [23]:
def anndata_to_sentence_files_hvg(
    sample,
    hvg_list: pd.Index,
    output_basedir: str,
    n_top_genes_in_sentence: int = 50
):
    """
    为单个HESTSample对象的所有spots生成独立的基因句子文件，但仅限于HVG。

    参数:
        sample (HESTSample): 要处理的样本对象。
        hvg_list (pd.Index): 全局高变基因列表。
        output_basedir (str): 保存所有句子文件夹的根目录。
        n_top_genes_in_sentence (int): 每个句子包含的基因数量。
    """
    # 为当前样本创建独立的输出目录
    sample_sentence_dir = os.path.join(output_basedir, f"{sample.sample_id}_sentences_hvg")
    os.makedirs(sample_sentence_dir, exist_ok=True)
    
    adata = sample.adata
    
    # 筛选AnnData，使其只包含高变基因，这样可以大大加速后续处理
    # .copy() 是为了避免修改原始adata对象
    adata_hvg = adata[:, adata.var_names.isin(hvg_list)].copy()
    
    if adata_hvg.n_vars == 0:
        print(f"警告: 样本 {sample.sample_id} 中没有找到任何全局HVGs，已跳过。")
        return

    obs_df = adata_hvg.obs.copy()
    gene_names_hvg = adata_hvg.var_names.to_numpy()
    
    # 检查并获取坐标
    if 'pxl_col_in_fullres' not in obs_df.columns or 'pxl_row_in_fullres' not in obs_df.columns:
        obs_df['pxl_col_in_fullres'] = adata_hvg.obsm['spatial'][:, 0]
        obs_df['pxl_row_in_fullres'] = adata_hvg.obsm['spatial'][:, 1]
        
    # 遍历每个spot并生成文件
    for i in tqdm(range(adata_hvg.n_obs), desc=f"处理 {sample.sample_id} (HVG)", leave=False):
        spot_info = obs_df.iloc[i]
        row_coord = int(round(spot_info['pxl_row_in_fullres']))
        col_coord = int(round(spot_info['pxl_col_in_fullres']))
        sentence_filename = os.path.join(sample_sentence_dir, f"{sample.sample_id}_{row_coord}_{col_coord}.txt")
        
        if os.path.exists(sentence_filename):
            continue

        # 获取当前spot在HVG子集上的表达向量
        expression_vector = adata_hvg.X[i].toarray().flatten() if issparse(adata_hvg.X) else np.array(adata_hvg.X[i]).flatten()
            
        # 在HVG子集内排序
        sorted_indices_hvg = np.argsort(expression_vector)[-1::-1]
        
        # 选择前 N 个基因（现在无需过滤管家基因，因为它们大概率不是HVG）
        top_genes = gene_names_hvg[sorted_indices_hvg[:n_top_genes_in_sentence]]
        
        # 写入文件
        with open(sentence_filename, 'w') as f:
            f.write(" ".join(top_genes))

In [42]:
# --- 1. 设置和数据加载 ---
data_dir = "/cwStorage/nodecw_group/jijh/hest_1k"
# 新的输出目录，以区分
sentence_output_basedir_hvg = "/cwStorage/nodecw_group/jijh/hest_sentences/"

print("初始化HEST数据集管理器...")
dataset = HESTDataset(data_dir=data_dir)
brain_samples = dataset.get_samples(organ="Brain", species='Mus musculus')
print(f"找到 {len(brain_samples)} 个Mus musculus大脑样本待处理。")
print("-" * 50)


初始化HEST数据集管理器...
找到 168 个Mus musculus大脑样本待处理。
--------------------------------------------------


In [25]:
from collections import Counter
import random

def report_varname_duplicates(samples):
    """
    对每个 HESTSample 中的 AnnData，检查 var_names 内部的重复项并打印出来。
    参数:
        samples (list): HESTSample 对象列表，要求每个 sample.adata 已经加载。
    """
    any_dup = False
    
    for sample in tqdm(samples, desc="检查基因名重复"):
        # 确保数据已加载
        if sample.adata is None:
            sample.load_st_data(lazy=True)
        adata = sample.adata
        if adata is None:
            print(f"[跳过] 样本 {sample.sample_id} 无法加载 AnnData。")
            continue

        names = list(adata.var_names)
        counts = Counter(names)
        # 找出在同一个 AnnData 里出现超过一次的基因名
        dups = [gene for gene, cnt in counts.items() if cnt > 1]
        
        if dups:
            any_dup = True
            print(f"\n⚠️ 样本 {sample.sample_id} 中共有 {len(dups)} 个重复基因名：")
            
            # 分析重复原因 - 随机选择几个重复基因进行详细分析
            sample_dups = random.sample(dups, min(3, len(dups)))
            
            for gene in sample_dups:
                duplicate_count = counts[gene]
                # 找到所有重复位置的索引
                duplicate_indices = [i for i, name in enumerate(names) if name == gene]
                
                print(f"  - 基因 '{gene}' 出现 {duplicate_count} 次，位置索引: {duplicate_indices}")
                
                # 检查这些重复位置的基因信息是否相同
                if hasattr(adata.var, 'gene_ids'):
                    gene_ids = [adata.var.iloc[idx]['gene_ids'] if 'gene_ids' in adata.var.columns else 'N/A' for idx in duplicate_indices]
                    print(f"    对应的基因ID: {gene_ids}")
                    if len(set(gene_ids)) > 1:
                        print(f"    ⚠️ 相同基因名对应不同的基因ID - 这可能是基因别名问题")
                    else:
                        print(f"    ✓ 相同基因名对应相同的基因ID - 这可能是数据处理过程中的重复")
            
            # 显示所有重复基因（如果列表太长，只显示前20个）
            print(f"  完整重复基因列表:")
            for g in dups[:20]:
                print(f"    - {g} (出现{counts[g]}次)")
            if len(dups) > 20:
                print(f"    ...（共 {len(dups)} 个重复基因，以上仅列出前 20 个）")

    if not any_dup:
        print("✅ 在所有样本内部均未发现重复的 var_names。")

# 执行检查
report_varname_duplicates(brain_samples)


检查基因名重复:   0%|          | 0/168 [00:00<?, ?it/s]

/public/home/jijh/micromamba/envs/imgprocess/lib/python3.12/site-packages/anndata/_core/anndata.py:1758: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")



⚠️ 样本 TENX131 中共有 6 个重复基因名：
  - 基因 'Arhgef4' 出现 2 次，位置索引: [95, 96]
  - 基因 'Aldoa' 出现 2 次，位置索引: [8355, 8357]
  - 基因 'Fam90a1b' 出现 2 次，位置索引: [18697, 18698]
  完整重复基因列表:
    - Arhgef4 (出现2次)
    - Pakap (出现2次)
    - Fam220a (出现2次)
    - Aldoa (出现2次)
    - Pcdha11 (出现2次)
    - Fam90a1b (出现2次)


In [ ]:

# --- 2. 计算全局高变基因 (HVGs) ---
# 这一步可能需要较长时间和较大内存，因为它会合并所有数据
# 如果内存不足，可以考虑从每个样本中随机抽取一部分spots来构建合并的adata

global_hvg_list = calculate_global_hvgs(
    samples_to_process=brain_samples,
    n_top_genes=3000, # 选择3000个HVGs是一个常见的SOTA实践
    sentence_output_basedir_hvg=sentence_output_basedir_hvg
)
print("-" * 50)



开始合并所有样本以计算全局HVGs...


加载并合并AnnData:   0%|          | 0/168 [00:00<?, ?it/s]

/public/home/jijh/micromamba/envs/imgprocess/lib/python3.12/site-packages/anndata/_core/anndata.py:1758: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")


发现重复基因名，正在处理样本中的 6 个重复基因...


/public/home/jijh/micromamba/envs/imgprocess/lib/python3.12/site-packages/anndata/_core/merge.py:1434: UserWarning: Only some AnnData objects have `.raw` attribute, not concatenating `.raw` attributes.
  warn(


所有样本已合并。总尺寸: (308159, 45646)


/public/home/jijh/micromamba/envs/imgprocess/lib/python3.12/site-packages/legacy_api_wrap/__init__.py:82: UserWarning: Some cells have zero counts
  return fn(*args_all, **kw)


成功识别出 3000 个全局高变基因。
--------------------------------------------------


In [ ]:
'Apoe' in global_hvg_list

ad_mouse_genes = [
    "App",    # Amyloid precursor protein
    "Psen1",  # Presenilin 1
    "Psen2",  # Presenilin 2
    "Mapt",   # Microtubule-associated protein tau
    "Apoe",   # Apolipoprotein E
    "Trem2",  # Triggering receptor expressed on myeloid cells 2
    "Clu",    # Clusterin
    "Picalm", # Phosphatidylinositol binding clathrin assembly protein
    "Bin1",   # Bridging integrator 1
    "Cd33",   # Siglec-3
    "Abca7",  # ATP-binding cassette sub-family A member 7
    "Sorl1",  # Sortilin-related receptor 1
    "Hcar2",  # Hydroxycarboxylic acid receptor 2
    "Vps35",   # Vacuolar protein sorting 35
    "Csf1r", # Colony stimulating factor 1 receptor
    "Bace1",    # β-分泌酶 1，关键的 Aβ 生成酶:contentReference[oaicite:0]{index=0}
    "Ide",      # 胰岛素降解酶，可降解 Aβ:contentReference[oaicite:1]{index=1}
    "Mme",      # Neprilysin，同样参与 Aβ 清除:contentReference[oaicite:2]{index=2}
    "Cd2ap",    # 调控内吞与 Aβ 清除:contentReference[oaicite:3]{index=3}
    "Cr1",      # 补体受体 1，免疫相关:contentReference[oaicite:4]{index=4}
    "Adam10",   # α-分泌酶，非淀粉样途径关键酶:contentReference[oaicite:5]{index=5}
    "Plcg2",    # 液泡内吞与信号传导:contentReference[oaicite:6]{index=6}
    "Spi1",     # 转录因子，免疫调控筛选中被鉴定:contentReference[oaicite:7]{index=7}
    "Mef2c",    # 神经发育转录因子，功能筛选中被鉴定:contentReference[oaicite:8]{index=8}
    "Gab2",     # 信号转导分子，GWAS 关联:contentReference[oaicite:9]{index=9}
    "Abcc11",   # ATP 结合盒家族成员，筛选中发现:contentReference[oaicite:10]{index=10}
    "ATCG1",    # 筛选中发现的新的候选基因:contentReference[oaicite:11]{index=11}
    "Ank3",     # GWAS 关联基因:contentReference[oaicite:12]{index=12}
    "Ms4a6a",   # MS4A 家族，调控免疫与脂质代谢:contentReference[oaicite:13]{index=13}
    "Agfg2",    # 涉及细胞信号转导:contentReference[oaicite:14]{index=14}
    "Cyc1",     # 线粒体功能相关:contentReference[oaicite:15]{index=15}
    "Hla-dra",  # 抗原呈递分子:contentReference[oaicite:16]{index=16}
    "Meg3",     # 长链非编码 RNA，表达差异:contentReference[oaicite:17]{index=17}
    "Mt2a",     # 金属硫蛋白，抗氧化应激:contentReference[oaicite:18]{index=18}
    "Ncald",    # 钙调节蛋白家族:contentReference[oaicite:19]{index=19}
    "Neu1",     # 神经氨酸苷酶 1，细胞内吞作用:contentReference[oaicite:20]{index=20}
    "Psmc3",    # 泛素-蛋白酶体系统:contentReference[oaicite:21]{index=21}
    "Serpinb6", # 丝氨酸蛋白酶抑制剂:contentReference[oaicite:22]{index=22}
    "Sparc",     # 细胞外基质蛋白，病理重塑:contentReference[oaicite:23]{index=23}
]

# Test what ad_mouse_genes are in global_hvg_list
ad_mouse_genes_in_hvg = [gene for gene in ad_mouse_genes if gene in global_hvg_list]
print(f"在全局HVG列表中找到 {len(ad_mouse_genes_in_hvg)} 个小鼠AD相关基因：")
print(ad_mouse_genes_in_hvg)

在全局HVG列表中找到 11 个小鼠AD相关基因：
['Apoe', 'Trem2', 'Clu', 'Bin1', 'Cd33', 'Csf1r', 'Cd2ap', 'Spi1', 'Mef2c', 'Ncald', 'Sparc']


In [41]:
# Add the mouse AD genes to the global HVG list
global_hvg_list = global_hvg_list.union(ad_mouse_genes_in_hvg)

In [43]:
sentence_output_basedir_hvg

'/cwStorage/nodecw_group/jijh/hest_sentences/'

In [ ]:
from concurrent.futures import ThreadPoolExecutor, ProcessPoolExecutor
import threading

# --- 3. 基于HVG列表，为每个样本生成句子文件 ---

print("开始批量生成基于HVG的基因句子文件...")

def process_single_sample(sample):
    """处理单个样本的函数"""
    try:
        # 确保数据已加载
        if sample.adata is None:
            sample.load_st_data(lazy=False)
            sample.adata = sample.adata.to_memory()
        if sample.adata is None:
            print(f"跳过样本 {sample.sample_id}，因为无法加载AnnData。")
            return
            
        anndata_to_sentence_files_hvg(
            sample=sample,
            hvg_list=global_hvg_list,
            output_basedir=sentence_output_basedir_hvg,
            n_top_genes_in_sentence=50
        )
        print(f"完成处理样本 {sample.sample_id}")
    except Exception as e:
        print(f"处理样本 {sample.sample_id} 时发生严重错误: {e}")

# 使用线程池执行器进行并行处理
with ThreadPoolExecutor(max_workers=10) as executor:
    # 提交所有任务
    futures = [executor.submit(process_single_sample, sample) for sample in brain_samples]
    
    # 等待所有任务完成并处理结果
    for future in tqdm(futures, desc="处理所有样本"):
        try:
            future.result()  # 这会等待任务完成并获取结果（如果有异常会抛出）
        except Exception as e:
            print(f"线程执行时发生错误: {e}")

print("\n所有基于HVG的基因句子已生成并保存到硬盘！")


开始批量生成基于HVG的基因句子文件...


处理 TENX88 (HVG):   0%|          | 0/5303 [00:00<?, ?it/s]

完成处理样本 TENX88


处理 TENX79 (HVG):   0%|          | 0/8462 [00:00<?, ?it/s]

完成处理样本 TENX79


...

完成处理样本 NCBI338


处理 NCBI337 (HVG):   0%|          | 0/188 [00:00<?, ?it/s]

完成处理样本 NCBI337


处理 NCBI336 (HVG):   0%|          | 0/214 [00:00<?, ?it/s]

完成处理样本 NCBI336


In [ ]:


# --- 4. 验证结果 ---
# 随机选择一个样本目录进行检查
if brain_samples:
    first_sample_id = brain_samples[0].sample_id
    sample_sentence_dir = os.path.join(sentence_output_basedir_hvg, f"{first_sample_id}_sentences_hvg")
    
    if os.path.exists(sample_sentence_dir) and os.listdir(sample_sentence_dir):
        print(f"\n--- 验证样本 {first_sample_id} 的结果 ---")
        sentence_files = os.listdir(sample_sentence_dir)
        random_sentence_file = random.choice(sentence_files)
        
        with open(os.path.join(sample_sentence_dir, random_sentence_file), 'r') as f:
            content = f.read()
            genes_in_sentence = content.split()
            print(f"随机抽取的句子文件 ({random_sentence_file}):")
            print(content)
            print(f"句子中的基因数量: {len(genes_in_sentence)}")
            # 检查这些基因是否都在HVG列表中
            are_all_hvgs = all(gene in global_hvg_list for gene in genes_in_sentence)
            print(f"句子中的所有基因是否都在全局HVG列表中? {'是' if are_all_hvgs else '否'}")
    else:
        print("未能找到生成的句子文件进行验证。")